In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
torch.cuda.empty_cache()
model_name = "Qwen/Qwen2-VL-2B-Instruct"
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16, 
    attn_implementation="eager",  
    device_map={"": "cuda"}
)
processor = AutoProcessor.from_pretrained(model_name,use_fast=True)

C:\Users\kmano\miniconda3\envs\lstr\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.41s/it]


In [2]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "C:/Users/kmano/Downloads/test.mp4",
                "max_pixels": 360 * 420,
                "fps": 1.0,
            },
            {"type": "text", "text": "explain this event in detailed manner."},
        ],
    }
]

In [3]:
messages

[{'role': 'user',
  'content': [{'type': 'video',
    'video': 'C:/Users/kmano/Downloads/test.mp4',
    'max_pixels': 151200,
    'fps': 1.0},
   {'type': 'text', 'text': 'explain this event in detailed manner.'}]}]

In [4]:
import time
st=time.time()
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")
generated_ids = model.generate(**inputs, max_new_tokens=512)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
end=time.time()
print("".join(output_text))
print("Time took to generate: ",end-st)

qwen-vl-utils using torchvision to read video.


The video depicts a military vehicle being destroyed by an explosion. The scene is set in a desolate, war-torn landscape with a dirt road and scattered debris. The vehicle is positioned on the road, and an explosion occurs, causing a large fireball to erupt from the vehicle's rear end. The explosion creates a significant amount of smoke and debris, which is visible in the video. The explosion appears to be a result of a military vehicle being targeted by an enemy, likely during a battle or conflict. The video provides a dramatic and intense visual representation of the destruction caused by military conflict.
Time took to generate:  25.13722014427185


In [ ]:
print("".join(output_text))

In [5]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [ ]:
!pip install -U bitsandbytes

In [ ]:
!pip install opencv-python pillow
